In [1]:
import os
import sys

PACKAGE_DIR = "/kaggle/src"
sys.path.append(PACKAGE_DIR)
sys.path.append(os.path.join(PACKAGE_DIR, "Penguin-ML-Library"))

In [2]:
import whoosh_utils
import whoosh_utils_penguin

Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


Processing /kaggle/input/whoosh-wheel-2-7-4/Whoosh-2.7.4-py2.py3-none-any.whl
Whoosh is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


In [3]:
import gc
import warnings

import numpy as np
import polars as pl
import yaml
from tqdm import tqdm

In [4]:
train_idx = whoosh_utils_penguin.load_index("/kaggle/input/rich-index")
searcher = whoosh_utils_penguin.get_searcher(train_idx)
qp = whoosh_utils_penguin.get_query_parser()

In [5]:
train_idx.doc_count()

200000

In [6]:
query = "id:US-2698435-A"
print(whoosh_utils_penguin.count_query_tokens(query=query))
results = whoosh_utils_penguin.execute_query(query, qp, searcher)[:5]
results

1


[]

In [10]:
test = pl.read_csv("/kaggle/input/uspto-boolean-search-optimization/test.csv")
test_patents = set(sum(test.to_numpy().tolist(), []))

exist = []
for patent in test_patents:
    query = f"id:{patent}"
    results = whoosh_utils_penguin.execute_query(query, qp, searcher)
    exist.append(len(results) > 0)
sum(exist) / len(exist)

0.0196078431372549

In [11]:
import json

with open("/kaggle/input/uspto-boolean-search-optimization/train_index_patent_ids.json", "r") as f:
    train_index_patent_ids = set(json.load(f))

exist = []
for patent in tqdm(train_index_patent_ids):
    query = f"id:{patent}"
    results = whoosh_utils_penguin.execute_query(query, qp, searcher)
    exist.append(len(results) > 0)
sum(exist) / len(exist)

100%|██████████| 200000/200000 [00:40<00:00, 4973.34it/s]


1.0

In [12]:
with open("/kaggle/input/patent2parquet/patent2parquet.json", "r") as f:
    patent2parquet = json.load(f)

patent2parquet = pl.DataFrame({"publication_number": list(patent2parquet.keys())})

exist = []
for patent in tqdm(patent2parquet["publication_number"][::100]):
    query = f"id:{patent}"
    results = whoosh_utils_penguin.execute_query(query, qp, searcher)
    exist.append(len(results) > 0)
sum(exist) / len(exist)

100%|██████████| 133077/133077 [00:16<00:00, 7860.05it/s]


0.015336985354343725